In [ ]:
import tensorflow as tf
import os
import cv2
import mediapipe as mp

input_path = 'input_video/'   # 좌표를 추출 할 수어 영상 데이터
output_path = 'output_text/'  # 좌표를 추출한 텍스트 파일을 저장할 경로

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3000)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야 한다.
        print(e)

In [ ]:
import os
import cv2
import mediapipe as mp

# 0번 GPU 할당
# GPU 환경에서 좌표 추출
with tf.device('/device:GPU:0'):  
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    PATH = os.getcwd()
    min_detection_confidence = 0.1  # 숫자가 작아질수록 좌표가 정확히 추출됩니다.
    min_tracking_confidence = 0.5

    def make_txt(folder, file_name, keypoints):
        output_data_path = output_path+ folder+'/'+ file_name[:-4] + '.txt'
        f = open(output_data_path, 'a', encoding="UTF-8")
        print('create file ', output_data_path)
        f.writelines(str(keypoints)[1:-1])
        f.close()

    def make_dir(input_data_path):
        if os.path.exists(input_data_path) and os.path.isdir(input_data_path):
            print('dir exists - ', input_data_path)
        else:
            os.mkdir(input_data_path)
            print('create dir', input_data_path)

    def track(input_data_path, folder, file_name):
        hands = mp_hands.Hands(min_detection_confidence=min_detection_confidence, 
                               min_tracking_confidence=min_tracking_confidence)
        print(input_data_path)
        cap = cv2.VideoCapture(input_data_path)

        keypoints = []
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break

            # Flip the image horizontally for a later selfie-view display, and convert
            # the BGR image to RGB.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

            # To improve performance, optionally mark the image as not writeable 
            # to pass by reference.
            image.flags.writeable = False
            results = hands.process(image)

            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                # 양 손에, 한 손당 21개의 좌표값이 추출됩니다.
                arow = []
                for hand_landmarks in results.multi_hand_landmarks:
                    for data_point in hand_landmarks.landmark:
                        nset = [data_point.x, data_point.y]
                        arow += nset
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                handed = len(results.multi_handedness)
                # 한 손만 사용하는 장면의 경우, 사용하지 않는 손의 좌표는 0으로 채웁니다.
                if handed == 1:
                    hd = results.multi_handedness[0].classification[0].label.split('\r')[0]
                    if hd=='left':
                        arow = arow + [0, 0]*21
                    else:
                        arow = [0, 0] * 21 + arow
                keypoints += arow
            # cv2.imshow('MediaPipe Hands', image) 
            if cv2.waitKey(5) & 0xFF == 27:
                break

        make_txt(folder, file_name, keypoints)
        hands.close()
        cap.release()

    def main(input_data_path):
        word_list = os.listdir(input_path)
        for word in word_list:
            file_list = os.listdir(input_path + "/" + word)
            make_dir(output_path + word)
            for file in file_list:
                track(input_path + '/' + word + '/' + file, word, file)

    if __name__ == "__main__":
        main(input_path)